# ETL Mini Project: GeneXpert Data Pipeline

This notebook simulates a real-world ETL process using **mock data** generated by the GeneXpert machine, which I’ve been handling in my current work.

The project demonstrates:

1. **Extracting** data from multiple file types (`CSV`, `XLSX`, `JSON`)
2. **Transforming** it (cleaning, standardizing, error handling)
3. **Loading** the cleaned dataset into a CSV file

All steps are logged in `etl_log.txt` for tracking and auditing purposes.

# Sample Raw Data

| Patient_ID | Patient_Code | Test_Type | Date_Tested | Status   | Result       |
|------------|--------------|-----------|------------|-----------|------------  |
| P1000      | GX-337       | HIV       | 2025-09-09 | Error     | 4017         |
| P1001      | GX-603       | HIV       | 2025-09-13 | Pending   | Invalid      |
| P1002      | GX-950       | MTB       | 2025-09-06 | Pending   | Not Detected |
| P1003      | GX-585       | CT_NG     | 2025-08-20 | Error     | 4017         |
| P1004      | GX-371       | MTB       | 2025-09-07 | Pending   | Not Detected |

## Note:
    - Contains duplicates across all raw data

In [1]:
# Importing libraries

import glob
import pandas as pd
from datetime import datetime
import os

etl_log = "etl_log.txt"
transformed_data_file = "clean_genexpert_data.csv"

In [2]:
# Creating a Logging Function for each proccesses' timestamp

def logging(message):
    timestamp_format = '[%Y-%m-%d %H:%M:%S]'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(etl_log, "a") as f:
        f.write(f"{timestamp} INFO: {message}\n")

In [3]:
# Creating Extraction Functions for each file Types

def extract_xlsx(xlsx):
    extracted_xlsx = pd.read_excel(xlsx)
    file_name = os.path.splitext(os.path.basename(xlsx))[0]
    extracted_xlsx["Source_File"] = file_name
    logging(f"Extracted {file_name}.xlsx file")
    return extracted_xlsx

def extract_csv(csv):
    extracted_csv = pd.read_csv(csv)
    file_name = os.path.splitext(os.path.basename(csv))[0]
    extracted_csv["Source_File"] = file_name
    logging(f"Extracted {file_name}.csv file")
    return extracted_csv

def extract_json(json_file):
    extracted_json_file = pd.read_json(json_file)
    file_name = os.path.splitext(os.path.basename(json_file))[0]
    extracted_json_file["Source_File"] = file_name
    logging(f"Extracted {file_name}.json file")
    return extracted_json_file

In [4]:
# Creating Combine Data Function to concatenate all extracted data (CSV/XLSX/JSON)

def extract_and_combine_dataframes():
    look_xlsx = glob.glob(r"raw_data\\*genexpert_data*.xlsx*")
    look_csv = glob.glob(r"raw_data\\*genexpert_data*.csv*")
    look_json = glob.glob(r"raw_data\\*genexpert_data*.json*")

    final_dfs = []
    for xlsx in look_xlsx:
        final_dfs.append(extract_xlsx(xlsx))
    for csv in look_csv:
        final_dfs.append(extract_csv(csv))
    for json_file in look_json:
        final_dfs.append(extract_json(json_file))

    combined_data = pd.concat(final_dfs, ignore_index=True)
    logging("Extracted XLSX/CSV/JSON files")
    return combined_data

In [5]:
# Creating Transform Function to transform the concatenated DataFrame into desired formats

def transform(combined_files):
    combined_files.columns = combined_files.columns.str.strip().str.lower().str.replace("_", " ")
    logging("Standardized Column Names")
    
    combined_files = combined_files.drop_duplicates(subset=["patient code", "test type", "result"])
    logging("Dropped duplicates based on 'patient code', 'test type', 'result'")
    
    combined_files["result"] = combined_files["result"].replace([5007, 4017, 2008, "5007", "4017", "2008"], "Error")
    logging("Replaced error codes as 'Error'")
    
    combined_files["date tested"] = pd.to_datetime(combined_files["date tested"])
    logging("Converted 'date tested' to datetime format")
    
    combined_files.sort_values(by="date tested", ascending=False, inplace=True)
    logging("Sorted data by date")
    
    return combined_files

In [6]:
# Creating Load Function for file saving

def loading(transformed):
    transformed.to_csv(transformed_data_file, index=False)

In [ ]:
# Running Extract, Transform, Load (ETL)

logging("ETL Started...")

logging("Data Extraction started")
extracted_data = extract_and_combine_dataframes()
logging("Data Extraction completed")

logging("Data Transformation started")
transformed_data = transform(extracted_data)
logging("Data Transformation completed")

logging("Saving Transformed Data")
loading(transformed_data)
logging("Saving Successful")

with open(etl_log, "r") as file:
    count_log_rows = len(file.readlines())

logging("Logging Completed...")
with open(etl_log, "a") as file:
    file.write(f"\nTotal Logs: {count_log_rows}\n")

## Transformed GeneXpert Data

This table shows the transformed mock data:

- Combined from multiple file types (CSV, XLSX, JSON)
- Standardized column names
- Duplicates removed based on `patient code`, `test type`, and `result`
- Error codes (5007, 4017, 2008) replaced with "Error"
- `date tested` converted to proper datetime format
- Sorted by `date tested` in descending order

| Patient_ID | Patient_Code | Test_Type | Date_Tested | Status   | Result       | Source_File      |
|------------|--------------|-----------|------------|-----------|----------    |------------------|
| P1013      | GX-349       | MTB       | 03/10/2025 | Error     | Error        | genexpert_data_4 |
| P1013      | GX-349       | MTB       | 03/10/2025 | Error     | Error        | genexpert_data_1 |
| P1005      | GX-731       | CT_NG     | 01/10/2025 | Error     | Error        | genexpert_data_5 |
| P1010      | GX-112       | HIV       | 01/10/2025 | Error     | Error        | genexpert_data_5 |
| P1002      | GX-975       | CT_NG     | 01/10/2025 | Pending   | Detected     | genexpert_data_2 |
| P1005      | GX-731       | CT_NG     | 01/10/2025 | Error     | Error        | genexpert_data_2 |
| P1010      | GX-112       | HIV       | 01/10/2025 | Error     | Error        | genexpert_data_2 |
| P1006      | GX-756       | MTB       | 01/10/2025 | Completed | Not Detected | genexpert_data_3 |
| P1005      | GX-673       | CT_NG     | 01/10/2025 | Completed | Invalid      | genexpert_data_1 |
| P1001      | GX-238       | CT_NG     | 30/09/2025 | Pending   | Invalid      | genexpert_data_2 |
| P1015      | GX-611       | HIV       | 27/09/2025 | Pending   | Invalid      | genexpert_data_3 |
| P1007      | GX-816       | HIV       | 26/09/2025 | Pending   | Detected     | genexpert_data_1 |
| P1014      | GX-197       | HIV       | 24/09/2025 | Pending   | Detected     | genexpert_data_3 |
| P1012      | GX-669       | HIV       | 23/09/2025 | Error     | Error        | genexpert_data_5 |


## ETL Process Log

The following log captures the step-by-step execution of the ETL process for the mock GeneXpert data. This includes data extraction, transformation, and saving of the cleaned dataset.

```
[2025-10-05 00:44:46] INFO: ETL Started...
[2025-10-05 00:44:46] INFO: Data Extraction started
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_1.xlsx file
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_2.xlsx file
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_3.xlsx file
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_4.csv file
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_5.csv file
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_6.csv file
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_7.json file
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_8.json file
[2025-10-05 00:44:47] INFO: Extracted genexpert_data_9.json file
[2025-10-05 00:44:47] INFO: Extracted XLSX/CSV/JSON files
[2025-10-05 00:44:47] INFO: Data Extraction completed
[2025-10-05 00:44:47] INFO: Data Transformation started
[2025-10-05 00:44:47] INFO: Standardized Column Names
[2025-10-05 00:44:47] INFO: Dropping duplicates based on 'patient code', 'test type', 'result'
[2025-10-05 00:44:47] INFO: Replaced error codes as 'Error'
[2025-10-05 00:44:47] INFO: Coverted 'date tested' to datetime format
[2025-10-05 00:44:47] INFO: Sorted data by date
[2025-10-05 00:44:47] INFO: Data Transformation completed
[2025-10-05 00:44:47] INFO: Saving Transformed Data
[2025-10-05 00:44:47] INFO: Saving Successful
[2025-10-05 00:44:47] INFO: Logging Completed...

Total Logs: 22
```

**Note:**  
This log provides a complete trace of the ETL process, including extraction of multiple file formats, data cleaning, transformation, and saving the final dataset. Each step is timestamped for tracking and debugging purposes.

## Accessing Data

You can access both the **raw data files** and the **cleaned dataset** used in this ETL mini-project:

- **Raw data files:** Located in the `raw_data/` folder. These are mock data files generated from GeneXpert machines I’ve been handling in my current work. They include XLSX, CSV, and JSON formats.
- **Cleaned dataset:** The final transformed and deduplicated data is saved as `clean_genexpert_data.csv`. This file has standardized column names, cleaned results, and sorted date-tested values.

> The cleaned dataset is ready for further analysis, visualization, or reporting.
